# Clustering Evaluation

Guilherme Franca (guifranca@gmail.com) 08/16/2015

Abstract: We report of the most usefull clustering evaluation metrics (at least from my perspective).

## Purity

Let $N$ be the number of data points, $\mathcal{C}=\{C_1, \dotsc, C_K \}$ the set of clusters, and $\mathcal{L} = \{  L_1, \dotsc, L_J \}$ the *true* labels of the data points. Then the purity coefficient is defined as
$$
\mbox{purity} = \dfrac{1}{N}\sum_{k=1}^K \max_{j\in J} | C_k \cap L_j |
$$
where $|A|$ denotes the number of elements in the set $A$. Notice that purity is in the range $[0,1]$. A high purity can be achieved by choosing a large number of clusters, and $1$ is achieved if $K=N$. However, this is an informative index provided we already had chosen the correct number of clusters $K$ (for instance by another analysis like elbow method, etc.).


## Adjusted Rand Index

Here we analyze whether pairs of points are correctly or wrongly classified to be in the same cluster or not. Two points should be assigned to the same cluster if they are similar, and should be assigned to different clusters if they are dissimilar. We define the following coefficients:

* True positive (TP): number of *similar* pairs assigned to the same cluster, i.e. the number of pairs in the same cluster in $\mathcal{C}$ and also in the same cluster in $\mathcal{L}$. These are the number of pairs correctly clustered together.
* True negative (TN): number of *dissimilar* pairs assigned to different clusters, i.e. number of pairs in different clusters in $\mathcal{C}$ and also in different clusters in $\mathcal{L}$. These are the number of pairs that are correctly assigned to different clusters, or the number of pairs correctly unclustered.
* False positive (FP): number of *dissimilar* pairs assigned to the same cluster, i.e. the number of pairs that are in the same cluster in $\mathcal{C}$ but in different clusters in $\mathcal{L}$. This is the error of clustering two dissimilar points together.
* False negative (FN): number of *similar* pairs assigned to different clusters, i.e. the number of pairs in different clusters in $\mathcal{C}$ but in the same cluster in $\mathcal{L}$. This is the error in *not* clustering similar points together.

The Rand index is then
$$
RI = \dfrac{TP + TN}{TP + FP + TN + FN}
$$

One issue with this index is that it gives the same weight to FP and FN, and separating similar points is usually worse than clustering dissimilar points, thus FN should have more weight than FP.

This index is only $0$ if $TP=TN=0$, which is something very unlikely. One can define the adjusted Rand index as follows. Let $N_{ij} = |C_i \cap L_j|$, and $N_i = \sum_j N_{ij}$
and $\tilde{N}_j = \sum_i N_{ij}$. Then
$$
ARI = \dfrac{RI - \mathbb{E}[RI]}{\max RI - \mathbb{E}[RI]}
$$
where the two partitions are picked at random subject to having the same number of classes and objects. Notice that this can be negative in the range $[-1,1]$.


## Mutual Information

The mutual information between partions $\mathcal{C}$ and $\mathcal{L}$ is given by
$$
\begin{aligned}
\mathbb{I}(\mathcal{C}, \mathcal{L}) &= \sum_{k=1}^K \sum_{j=1}^J
p(C_k\cap L_j) \log \left[ \dfrac{p(C_k \cap L_j)}{p(C_k) p(L_j)}  \right] \\
&= \sum_{k=1}^K \sum_{j=1}^J \dfrac{|C_k \cap L_j|}{N} \log \left[ N \dfrac{|C_k\cap L_j|}{|C_k| |L_j|} \right]
\end{aligned}
$$
This is in the range between $0$ and $\min\{ \mathbb{H}(\mathcal{C}),\mathbb{H}(\mathcal{L}) \}$ where the entropies are given by
$$
\mathbb{H}(\mathcal{C}) = - \sum_{k=1}^K p(C_k) \log p(C_k) = -\sum_{k=1}^K \dfrac{|C_k|}{N} \log \dfrac{|C_k|}{N}
$$
and analogously for $\mathcal{L}$. A large value of $\mathbb{I}$ can be achieved by choosing a large number of clusters. Since the entropy increases with the number of clusters we defined the *normalized mutual information* by
$$
NMI(\mathcal{C},\mathcal{L}) = \dfrac{\mathbb{I}(\mathcal{C},\mathcal{L})}{(\mathbb{H}(
\mathcal{C})+\mathbb{H}(\mathcal{L}))/2}
$$
and this coefficient is between $0$ and $1$.

## Implementation

The scikit-learn library have these indexes already implemented. It seems however that they define the mutual information as
$$
NMI(\mathcal{C},\mathcal{L}) = \dfrac{\mathbb{I}(\mathcal{C},\mathcal{L})}{
\sqrt{\mathbb{H}(\mathcal{C}) \, \mathbb{H}(\mathcal{L})} }
$$
They also have the *Adjusted Mutual information*
$$
AMI(\mathcal{C},\mathcal{L}) = \dfrac{\mathbb{I}-\mathbb{E}[\mathbb{I}]}{
\max\{ \mathbb{H}(\mathcal{C}),\mathbb{H}(\mathcal{L}) \} - \mathbb{E}[\mathbb{I}]}
$$
which is normalized against chance.

In [1]:
from sklearn import metrics

In [19]:
# Adjusted Rand index
# this ignores permutations, and we can rename the elements, and this is symmetric
c = [0,0,1,1,1,1]
l = [1,1,1,0,0,0]
metrics.adjusted_rand_score(l, c)

0.32432432432432434

In [28]:
c = [0,0,1,1,1,1]
l = [0,0,0,0,1,1]
print metrics.mutual_info_score(l, c)
print metrics.normalized_mutual_info_score(l, c)
print metrics.adjusted_mutual_info_score(l, c)
print metrics.v_measure_score(l, c)

0.174416047922
0.274017542121
0.0903702202995
0.274017542121
